# Accelerating Python

Main reference:
- [*Numerical Python*](https://link.springer.com/book/10.1007/978-1-4842-4246-9), Robert Johansson, Chapter 19: *Code Optimization*

In [ ]:
import numpy as np

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import numba

[`Numba`](https://numba.pydata.org)
=================================

In [ ]:
def py_sum(data: list):
    """Pythonic-function: sum
    """
    s = 0
    for d in data:
        s += d
    return s

In [ ]:
repeat = 50000
data = np.random.randn(repeat)

In [ ]:
# profile to output
# following https://stackoverflow.com/questions/17310752/can-you-capture-the-output-of-ipythons-magic-methods-timeit
t_py = %timeit -o py_sum(data)

In [ ]:
dir(t_py)

In [ ]:
t_py.average

In [ ]:
t_np = %timeit -o np.sum(data)

In [ ]:
def times(a: float, b: float):
    """return the ratio of average runtime of a and b"""
    t = a.average / b.average
    print("ratio of average runtime: ", round(t))

In [ ]:
times(t_py, t_np)

Check if py_sum and np.sum return the **equal** results *within given tolerence*

In [ ]:
tolerence = 1e-10
assert abs(py_sum(data) - np.sum(data)) < tolerence

`numba.jit` Decorator
---------------------
See [PEP 318](https://www.python.org/dev/peps/pep-0318/) -- Decorators for Functions and Methods for details about the "decorator".

In [ ]:
@numba.jit
def jit_sum(data):
    s = 0
    for d in data:
        s += d
    return s

In [ ]:
t_jit = %timeit -o jit_sum(data)

In [ ]:
times(t_py, t_jit)

Decorate an defined function
----------------------------

In [ ]:
jit_sum2 = numba.jit()(py_sum)

In [ ]:
t_jit2 = %timeit -o jit_sum2(data)

In [ ]:
t_jit2.average

Check equality within the given tolerence

In [ ]:
assert abs(py_sum(data) - jit_sum(data)) < tolerence

Julia fractal
---------
The [*Julia set*](https://brilliant.org/wiki/fractals/) is defined as the set of all complex numbers, $z$, which arebounded under repeated iteration of the complex quadratic polynomial
$$
z_{n+1} = z_n^2 + c
$$
for a constant complex number $c$.

In [ ]:
def py_julia_fractal(z_re, z_im, j):
    """Pythonic function of Julia fractal"""
    c = -0.05 + 0.68j
    for m in range(len(z_re)):
        for n in range(len(z_im)):
            z = z_re[m] + 1j * z_im[n]
            for t in range(256):
                z = z ** 2 + c
                if np.abs(z) > 2.0:
                    j[m, n] = t
                    break

In [ ]:
N = 1024
z_real = np.linspace(-1.5, 1.5, N)
z_imag = np.linspace(-1.5, 1.5, N)

In [ ]:
j = np.zeros((N, N), np.int64)
t_py_julia = %timeit -n1 -r1 -o py_julia_fractal(z_real, z_imag, j)

In [ ]:
jit_julia_fractal = numba.jit(nopython=True)(py_julia_fractal)

In [ ]:
j = np.zeros((N, N), np.int64)
t_jit_julia = %timeit -o jit_julia_fractal(z_real, z_imag, j)

In [ ]:
times(t_py_julia, t_jit_julia)

`numba.jit(nopython=True)` is equivalent to `numba.njit()`

In [ ]:
njit_julia_fractal = numba.njit()(py_julia_fractal)

In [ ]:
j = np.zeros((N, N), np.int64)
t_njit_julia = %timeit -o jit_julia_fractal(z_real, z_imag, j)

In [ ]:
times(t_jit_julia, t_njit_julia)

In [ ]:
fig, ax =plt.subplots(figsize=(8, 8))
ax.imshow(j, cmap=plt.cm.RdBu_r, extent=[-1.5, 1.5, -1.5, 1.5])
ax.set_xlabel("$\mathrm{Re}(z)$", fontsize=12)
ax.set_ylabel("$\mathrm{Im}(z)$", fontsize=12)

Heaviside step function:
\begin{equation}
\Theta(x) =
\begin{cases}
0, \quad & x < 0 \\
1/2, & x = 0 \\
1, & x > 0
\end{cases}
\end{equation}

`numba.vectorize`
-----------------

In [ ]:
def py_Heaviside(x: float):
    """Heaviside step function"""
    if x == 0.0:
        return 0.5
    elif x < 0.0:
        return 0.0
    else:
        return 1.0

In [ ]:
x = np.linspace(-2, 2, 50001)
t_py_H = %timeit -o [py_Heaviside(xx) for xx in x]

In [ ]:
np_vec_Heaviside = np.vectorize(py_Heaviside)
t_np_H = %timeit -o np_vec_Heaviside(x)

In [ ]:
def np_Heaviside(x):
    """x: NumPy array"""
    return (x > 0.0) + (x == 0.0) / 2.0

In [ ]:
t_npa_H = %timeit -o np_Heaviside(x)

In [ ]:
times(t_np_H, t_npa_H)

In [ ]:
@numba.vectorize([numba.float32(numba.float32),
                  numba.float64(numba.float64)])
def jit_Heaviside(x):
    if x == 0.0:
        return 0.5
    if x < 0:
        return 0.0
    else:
        return 1.0

In [ ]:
t_jit_H = %timeit -o jit_Heaviside(x)

In [ ]:
times(t_np_H, t_jit_H)

In [ ]:
jit_Heaviside(x[24990:25010])

Fastmath
--------
Pease efer to https://numba.pydata.org/numba-doc/latest/user/performance-tips.html for other performance tips.

In [ ]:
njit_f_H = numba.njit(fastmath=True)(np_Heaviside)

In [ ]:
t_njit_f_H = %timeit -o njit_f_H(x)

[`Cython`](https://cython.org/)
=============================

In [ ]:
%load_ext Cython

In [ ]:
%%cython

def cy_sum(data):
    s = 0.0
    for d in data:
        s += d
    return s

In [ ]:
t_cy = %timeit -o cy_sum(data)

In [ ]:
times(t_py, t_cy)

In [ ]:
%%cython
cimport numpy
cimport cython

@cython.boundscheck(False)
@cython.wraparound(False)
def cy_sum2(numpy.ndarray[numpy.float64_t, ndim=1] data):
    cdef numpy.float64_t s = 0.0
    cdef int n, N = len(data)
    for n in range(N):
        s += data[n]
    return s

In [ ]:
t_cy2 = %timeit -o cy_sum2(data)

In [ ]:
times(t_py, t_cy2)

Summary
=======

- `Numba` 几乎不必修改原 `Python` 代码，即可实现对特定 `Python` 函数的优化，特别是与 `for` 循环和 `NumPy` 相关的优化
- `Cython` 在几乎不修改原 `Python` 代码的基础上，优化幅度较小；为得到较好的优化，需要对原 `Python` 代码进行较大的修改